In [1]:
%run Functions.ipynb

In [3]:
# Rules.ipynb
from datetime import datetime, timedelta
import pandas as pd
from sqlalchemy import text
import json
from typing import List

RULE_REGISTRY = {}

def rule(rule_id):
    def decorator(func):
        RULE_REGISTRY[rule_id] = func
        return func
    return decorator

def store_rule_output(rule_id: str, df: pd.DataFrame):
    engine = get_db_engine()
    json_data = df.to_json(orient='records')
    with engine.connect() as conn:
        conn.execute(text("""
            INSERT INTO rule_results (rule_id, result_json, last_updated)
            VALUES (:rid, :rjson, now())
            ON CONFLICT (rule_id) DO UPDATE SET 
                result_json = EXCLUDED.result_json,
                last_updated = EXCLUDED.last_updated
        """), {"rid": rule_id, "rjson": json_data})

def get_cached_rule_output(rule_id: str):
    engine = get_db_engine()
    with engine.connect() as conn:
        result = conn.execute(text("SELECT result_json FROM rule_results WHERE rule_id = :rid"), {"rid": rule_id})
        row = result.fetchone()
        return json.loads(row[0]) if row else None

# ---------------------------
# RULE IMPLEMENTATIONS
# ---------------------------

@rule("rule_01")
def customers_without_accounts():
    df = get_cust_acc()
    result = df[df['account_no'].isnull()]
    store_rule_output("rule_01", result)
    return result

@rule("rule_02")
def accounts_without_transactions():
    df = get_acc_tran()
    result = df[df['transaction_id'].isnull()]
    store_rule_output("rule_02", result)
    return result

@rule("rule_03")
def high_value_transactions():
    df = get_transactions(min_amount=100000)
    store_rule_output("rule_03", df)
    return df

@rule("rule_04")
def old_accounts():
    df = get_accounts()
    result = df[pd.to_datetime(df['activation_date']) < datetime(2020, 1, 1)]
    store_rule_output("rule_04", result)
    return result

@rule("rule_05")
def zero_amount_transactions():
    df = get_transactions(amount=0.0)
    store_rule_output("rule_05", df)
    return df

@rule("rule_06")
def customers_with_multiple_accounts():
    df = get_accounts()
    grouped = df.groupby('customer_id').filter(lambda x: len(x) > 1)
    store_rule_output("rule_06", grouped)
    return grouped

@rule("rule_07")
def dormant_accounts():
    df = get_acc_tran()
    one_year_ago = datetime.now() - timedelta(days=365)
    result = df[pd.to_datetime(df['transaction_time']) < one_year_ago]
    result = result.drop_duplicates('account_no')
    store_rule_output("rule_07", result)
    return result

@rule("rule_08")
def transactions_on_inactive_accounts():
    df = get_acc_tran()
    result = df[df['account_status'].str.lower() != 'active']
    store_rule_output("rule_08", result)
    return result

@rule("rule_09")
def missing_city_customers():
    df = get_customers()
    result = df[df['city'].isnull() | (df['city'].str.strip() == '')]
    store_rule_output("rule_09", result)
    return result

@rule("rule_10")
def transaction_before_activation():
    df = get_acc_tran()
    df['transaction_time'] = pd.to_datetime(df['transaction_time'])
    df['activation_date'] = pd.to_datetime(df['activation_date'])
    result = df[df['transaction_time'] < df['activation_date']]
    store_rule_output("rule_10", result)
    return result

@rule("rule_11")
def burst_transactions():
    df = get_transactions()
    df['transaction_time'] = pd.to_datetime(df['transaction_time'])
    grouped = df.groupby(['account_no', pd.Grouper(key='transaction_time', freq='1min')]).filter(lambda x: len(x) >= 5)
    store_rule_output("rule_11", grouped)
    return grouped

@rule("rule_12")
def frequent_small_transactions():
    df = get_transactions(max_amount=100)
    grouped = df.groupby('account_no').filter(lambda x: len(x) >= 10)
    store_rule_output("rule_12", grouped)
    return grouped

@rule("rule_13")
def customer_transaction_multiple_accounts():
    df = get_cust_acc_tran()
    result = df.groupby('customer_id').filter(lambda x: x['account_no'].nunique() > 1 and len(x) > 5)
    store_rule_output("rule_13", result)
    return result

@rule("rule_14")
def mismatched_customer_ids():
    df = get_cust_acc_tran()
    result = df[df['customer_id'] != df['customer_id']]
    store_rule_output("rule_14", result)
    return result

@rule("rule_15")
def potential_duplicates_by_name():
    df = get_customers()
    dupes = df[df.duplicated(subset=['name'], keep=False)]
    store_rule_output("rule_15", dupes)
    return dupes

def run_rule(rule_id: str) -> List[dict]:
    cached = get_cached_rule_output(rule_id)
    if cached:
        return cached
    result_df = RULE_REGISTRY[rule_id]()
    return json.loads(result_df.to_json(orient='records'))


In [26]:
#custom rules